## Notebook settings

In [4]:
%load_ext autoreload
%autoreload 2

%autosave 10

%load_ext lab_black

Autosaving every 10 seconds


In [5]:
import sys
import os

sys.path.insert(0, os.path.abspath(os.path.join("..")))

## Imports

In [6]:
from lfp_analysis.data import *
from lfp_analysis.process import *
from lfp_analysis.resnet import *
from lfp_analysis.svm import *

from fastai.vision import *
import torch.nn.functional as F
from torchvision.transforms import ToPILImage, ToTensor

/home/fernandorp/anaconda3/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729096996/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [7]:
import numpy as np
import pandas as pd
import h5py

from pathlib import Path

import matplotlib.pyplot as plt

%matplotlib widget

# Import data:

In [8]:
DATA_PATH = Path("./../data")
ENV = "ET1_Pegboard_OFF"

H5_SOURCE = DATA_PATH / "TF_and_df" / f"{ENV}.h5"
CSV_SOURCE = DATA_PATH / "TF_and_df" / f"{ENV}.csv"

### Read TF

In [9]:
h5 = h5py.File(H5_SOURCE, "r")
TF = h5["TF"][:]
# h5.close()

In [10]:
TF.shape

(4, 99, 1455948)

### Read df_data

In [100]:
df_data = pd.read_csv(CSV_SOURCE, index_col=0)

In [101]:
df_data["label"].value_counts(normalize=True)

False    0.538948
True     0.461052
Name: label, dtype: float64

# Baseline Classifier:

In [102]:
BLClassifier(TF, df_data).classify()

On Train: 
              precision    recall  f1-score   support

       False       0.85      0.77      0.81      2469
        True       0.76      0.84      0.80      2080

    accuracy                           0.80      4549
   macro avg       0.80      0.81      0.80      4549
weighted avg       0.81      0.80      0.80      4549

[[1903  566]
 [ 329 1751]]

On Valid:
              precision    recall  f1-score   support

       False       0.62      0.57      0.59       596
        True       0.57      0.62      0.59       542

    accuracy                           0.59      1138
   macro avg       0.59      0.59      0.59      1138
weighted avg       0.60      0.59      0.59      1138

[[339 257]
 [206 336]]


(0.8032534622994064, 0.5931458699472759)

# Prepare learner:

## Dataloader prep

### Data stream:

In [103]:
def get_x(row):
    return torch.tensor(TF[:, :, row["id_start"] : row["id_end"]])


def get_y(row):
    return row["label"]


def splitter(df):
    train = df.index[df["is_valid"] == 0].tolist()
    valid = df.index[df["is_valid"] == 1].tolist()
    return train, valid

### Data transforms:

In [104]:
class Resizer(Transform):
    def __init__(self, size):
        self.size = size

    def encodes(self, X):
        if isinstance(X, TensorCategory):
            return X
        return torch.stack(
            [ToTensor()(ToPILImage()(x).resize(self.size)) for x in X]
        ).squeeze()


class LFPNormalizer(Transform):
    def __init__(self, means, stds):
        self.means, self.stds = means, stds

    def encodes(self, X):
        if isinstance(X, TensorCategory):
            return X
        xs = torch.unbind(X, 1)

        return torch.stack(
            [(x - self.means[ii]) / self.stds[ii] for ii, x in enumerate(xs)], 1
        )

In [105]:
def LFP_block():
    return TransformBlock(
        item_tfms=[Resizer((160, 160)), IntToFloatTensor],
        batch_tfms=LFPNormalizer([0.5, 0.5, 0.5, 0.5], [0.098, 0.098, 0.098, 0.098]),
    )

### Dataloader:

In [106]:
dblock = DataBlock(
    blocks=(LFP_block, CategoryBlock), get_x=get_x, get_y=get_y, splitter=splitter,
)

In [108]:
# dblock.summary(df_data)

In [109]:
dls = dblock.dataloaders(df_data, bs=64)

# Learner

In [110]:
resnet = ResNet(4, 2, [2, 3, 4, 3], 4)

In [111]:
learn = Learner(
    dls,
    resnet,
    metrics=[accuracy],
    loss_func=F.cross_entropy,
    # cbs=[ActivationStats(with_hist=True)],
)

In [112]:
xb, yb = dls.one_batch()
yb.shape, xb.shape

(torch.Size([64]), torch.Size([64, 4, 160, 160]))

In [113]:
init_loss = learn.loss_func(learn.model(xb), yb)
init_loss

TensorCategory(0.7339, grad_fn=<AliasBackward>)

In [114]:
del xb, yb

In [115]:
learn.lr_find()

OSError: [Errno 12] Cannot allocate memory

In [ ]:
learn.fit_one_cycle(10, 2*10e-5)